# Jupyter notebook for downloading Surface Albedo data

Surface albedo is critical for understanding the Earth’s climate, its variability, and trends, as it quantifies the fraction of solar irradiance reflected by the Earth’s surface. This variable plays a vital role in the global radiation budget, influencing surface temperatures, water balance, and climate feedback mechanisms, such as the ice-albedo feedback in polar regions.

Surface albedo represents the proportion of incoming solar radiation reflected by the surface under direct illumination conditions, computed for local solar noon. It depends on surface properties such as land cover, snow and ice extent, soil moisture, and vegetation characteristics. Surface albedo is essential for climate and weather studies, as it governs the amount of solar energy absorbed by the Earth, impacting energy exchange between the surface and atmosphere.

Surface albedo estimates are derived at a 10-day frequency using a 20-day moving temporal window of daily surface reflectance data. The processing algorithm normalises reflectances through inversion of a semi-empirical, kernel-driven reflectance model, prioritising recent observations. Directional albedo is computed per spectral band by integrating over the viewing hemisphere, with broadband albedo derived via a linear relationship from spectral albedo. Earlier versions of the dataset utilised data from NOAA/AVHRR (4km spatial resolution), SPOT-VGT, and PROBA-V (1km resolution), with products available from 1981 to the present. 



## Content of this notebook

This notebook explains how you can get access to EMCWF data and how you can download a custom-tailored subset of this dataset.

For the purpose of this assignment, only a specific variable and a delimited time span will be chosen, however this is scalable. 

The dataset has the following characteristics: 

* Data type: Gridded
* Projection: Plate Carrée
* Horizontal coverage: Global land surface
* Horizontal resolution: AVHRR: 1/30° (\~4 km), VGT: 1/112° (\~1 km), Sentinel-3: 1/336° (~300 m)
* Vertical coverage: Top of the canopy
* Temporal coverage: AVHRR: September 1981 to December 2005, SPOT-VGT: April 1998 to May 2014, PROBA-VGT: October 2013 to June 2020, Sentinel-3: July 2018 to December 2024
* Temporal resolution: 10 days
* File format: NetCDF

For more information read through the [Surface albedo documentation](https://cds.climate.copernicus.eu/datasets/satellite-albedo?tab=documentation).




## How to proceed

1. 

2. First register in the ESGF MetaGrid. Copy and store your key??

2. Install the client. Use pip or anaconda, depending on your specific case. 

3. Browse the catalogue to find the dataset you want to download. Note the fields of the specific query. 


1. Read through the [ERA5 documentation](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-complete?tab=form) to understand the data structure and identify which data/access point we need.
2. Install the [CDS API] (for Anaconda use : conda install -c conda-forge cdsapi)
   * But first, make sure you are registered at [Copernicus Data Store](https://accounts.ecmwf.int/auth/realms/ecmwf/protocol/openid-connect/auth?client_id=cds&scope=openid%20email&response_type=code&redirect_uri=https%3A%2F%2Fcds.climate.copernicus.eu%2Fapi%2Fauth%2Fcallback%2Fkeycloak&state=eSFj1D686DwLjXr9KQRyDdS8UA1dAVnzsJzSvXQTetM&code_challenge=nf66xbC6c-X_rt1RIoT4al-fNscBETZhfSzP0i1DSbc&code_challenge_method=S256).
   * Next, copy and store your API key in file `$HOME/.cdsapirc`. The key can be found at the bottom of your personal profile when you are logged in to the CDS. The file should be created in the HOME location. Format of the `.cdsapirc` file:

```
url: https://cds.climate.copernicus.eu/api
key: {your-api-key-here}
```

5. Browse through the [ERA5 data catalogue](https://apps.ecmwf.int/data-catalogues/era5/?class=ea) and select the fields you want to download.
6. After composing your search, click on "Show API request" and copy the commands into your notebook.


 Select Netcdf as output format.


In [23]:
import sys 
local_lib = '/p/home/jusers/schultz1/jsccloud/.local/lib/python3.12/site-packages/'

print(local_lib in sys.path)
if not (local_lib in sys.path):
    sys.path.append(local_lib)

True


In [27]:
import cdsapi

dataset = "satellite-albedo"
request = {
    "variable": ["albb_bh"],
    "satellite": ["noaa_14"],
    "sensor": "avhrr",
    "horizontal_resolution": ["4km"],
    "product_version": ["v2"],
    "year": ["1999"],
    "month": ["07"],
    "nominal_day": [
        "10",
        "20",
        "31"
    ],
    "area": [90, -180, -90, 180], 
     "format": "netcdf"
}

client = cdsapi.Client()
client.retrieve(dataset, request, 'Sentinel3_SurfaceAlbedo.nc')




2025-12-02 22:00:27,904 INFO Request ID is b22798ce-5d6f-4563-8898-ffda03100068
INFO:ecmwf.datastores.legacy_client:Request ID is b22798ce-5d6f-4563-8898-ffda03100068
2025-12-02 22:00:28,007 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-12-02 22:00:41,691 WARNING Download format not supported for this dataset. Defaulting to zip.
2025-12-02 22:00:41,693 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-12-02 22:02:22,332 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful


6845c08fbc18bbbbe248ac020247035d.zip:   0%|          | 0.00/287M [00:00<?, ?B/s]

'Sentinel3_SurfaceAlbedo.nc'

C:\Users\nagib
